<a href="https://colab.research.google.com/github/Liekeverhoeven262/Master-Thesis-Data-Science-Society/blob/main/Final_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Construct the path to the file in Google Drive
personal_data_path = '/content/drive/My Drive/Master Thesis/Data/FINAL_personal_data_compact3without.xlsx'
thermal_data_path = '/content/drive/My Drive/Master Thesis/Data/FINAL_scaled_features_df_max.xlsx'

# Load the dataset
personal_data_df = pd.read_excel(personal_data_path)
thermal_data_df = pd.read_excel(thermal_data_path)

In [ ]:
print(personal_data_df.columns)

Index(['ID', 'Cluster', 'Sum2', 'VVR1', 'VVR2'], dtype='object')


In [ ]:
print(personal_data_df.head)

<bound method NDFrame.head of       ID  Cluster  Sum2  VVR1  VVR2
0      5        2     0    13    12
1      6        1     0     9    10
2      7        1     1    12    14
3      8        0     0    10    10
4      9        0     0     9    10
..   ...      ...   ...   ...   ...
310  328        0     0    11    12
311  329        0     0    11    10
312  330        0     0    12    14
313  331        1     0    15    16
314  332        1     1    15    18

[315 rows x 5 columns]>


In [ ]:
print(thermal_data_df.columns)

Index(['ID', 'Left Cheek Sum', 'Left Cheek Variance',
       'Left Cheek Standard Deviation', 'Left Cheek Maximum',
       'Left Cheek Minimum', 'Left Cheek Median', 'Left Cheek Mean',
       'Left Cheek Mean Absolute Change', 'Left Cheek Maximum Slope',
       'Left Cheek Minimum Slope', 'Right Cheek Sum', 'Right Cheek Variance',
       'Right Cheek Standard Deviation', 'Right Cheek Maximum',
       'Right Cheek Minimum', 'Right Cheek Median', 'Right Cheek Mean',
       'Right Cheek Mean Absolute Change', 'Right Cheek Maximum Slope',
       'Right Cheek Minimum Slope', 'Below Nose Sum', 'Below Nose Variance',
       'Below Nose Standard Deviation', 'Below Nose Maximum',
       'Below Nose Minimum', 'Below Nose Median', 'Below Nose Mean',
       'Below Nose Mean Absolute Change', 'Below Nose Maximum Slope',
       'Below Nose Minimum Slope', 'Between Eyes Sum', 'Between Eyes Variance',
       'Between Eyes Standard Deviation', 'Between Eyes Maximum',
       'Between Eyes Minimum', 'Bet

In [ ]:
data = pd.merge(thermal_data_df, personal_data_df, on='ID')

In [ ]:
print(data.columns)

Index(['ID', 'Left Cheek Sum', 'Left Cheek Variance',
       'Left Cheek Standard Deviation', 'Left Cheek Maximum',
       'Left Cheek Minimum', 'Left Cheek Median', 'Left Cheek Mean',
       'Left Cheek Mean Absolute Change', 'Left Cheek Maximum Slope',
       'Left Cheek Minimum Slope', 'Right Cheek Sum', 'Right Cheek Variance',
       'Right Cheek Standard Deviation', 'Right Cheek Maximum',
       'Right Cheek Minimum', 'Right Cheek Median', 'Right Cheek Mean',
       'Right Cheek Mean Absolute Change', 'Right Cheek Maximum Slope',
       'Right Cheek Minimum Slope', 'Below Nose Sum', 'Below Nose Variance',
       'Below Nose Standard Deviation', 'Below Nose Maximum',
       'Below Nose Minimum', 'Below Nose Median', 'Below Nose Mean',
       'Below Nose Mean Absolute Change', 'Below Nose Maximum Slope',
       'Below Nose Minimum Slope', 'Between Eyes Sum', 'Between Eyes Variance',
       'Between Eyes Standard Deviation', 'Between Eyes Maximum',
       'Between Eyes Minimum', 'Bet

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from imblearn.over_sampling import SMOTE

In [ ]:
# Feature columns: thermal data and VVR1, VVR2
features = data.loc[:, 'Left Cheek Sum':'Nose Minimum Slope'].columns.tolist()
features.extend(['VVR1', 'VVR2'])

# Target variable
target = 'Sum2'

In [ ]:
print(features)

['Left Cheek Sum', 'Left Cheek Variance', 'Left Cheek Standard Deviation', 'Left Cheek Maximum', 'Left Cheek Minimum', 'Left Cheek Median', 'Left Cheek Mean', 'Left Cheek Mean Absolute Change', 'Left Cheek Maximum Slope', 'Left Cheek Minimum Slope', 'Right Cheek Sum', 'Right Cheek Variance', 'Right Cheek Standard Deviation', 'Right Cheek Maximum', 'Right Cheek Minimum', 'Right Cheek Median', 'Right Cheek Mean', 'Right Cheek Mean Absolute Change', 'Right Cheek Maximum Slope', 'Right Cheek Minimum Slope', 'Below Nose Sum', 'Below Nose Variance', 'Below Nose Standard Deviation', 'Below Nose Maximum', 'Below Nose Minimum', 'Below Nose Median', 'Below Nose Mean', 'Below Nose Mean Absolute Change', 'Below Nose Maximum Slope', 'Below Nose Minimum Slope', 'Between Eyes Sum', 'Between Eyes Variance', 'Between Eyes Standard Deviation', 'Between Eyes Maximum', 'Between Eyes Minimum', 'Between Eyes Median', 'Between Eyes Mean', 'Between Eyes Mean Absolute Change', 'Between Eyes Maximum Slope', 'Be

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data[features], data[target], test_size=0.2, random_state=42)

# Apply SMOTE only on the training data
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Checking the balance of the classes after SMOTE
print(pd.Series(y_train_smote).value_counts())

Sum2
0    168
1    168
Name: count, dtype: int64


In [ ]:
from sklearn.metrics import classification_report

## Model 1

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the model
rf = RandomForestClassifier(random_state=42)

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [10, 15, 20, 25, None],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 6]
}

# Setup the GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='recall', n_jobs=-1)

# Fit GridSearchCV
grid_search.fit(X_train_smote, y_train_smote)

# Best parameters and best recall
print("Best parameters:", grid_search.best_params_)
print("Best recall:", grid_search.best_score_)

Best parameters: {'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
Best recall: 0.8684491978609625


In [ ]:
from sklearn.metrics import classification_report, recall_score, precision_score, average_precision_score

# Setup the optimized Random Forest model using the best parameters from GridSearchCV
optimized_rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_leaf=1,
    min_samples_split=2,
    random_state=42
)

# Train the model on the balanced training set
optimized_rf.fit(X_train_smote, y_train_smote)

# Predict on the test data
predictions_optimized = optimized_rf.predict(X_test)
probabilities_optimized = optimized_rf.predict_proba(X_test)[:, 1]

# Evaluate the model
print("Optimized Model Evaluation:")
print(classification_report(y_test, predictions_optimized))

# Calculate specific metrics
precision = precision_score(y_test, predictions_optimized, pos_label=1)
recall = recall_score(y_test, predictions_optimized, pos_label=1)
auc_pr = average_precision_score(y_test, probabilities_optimized)
print(f'Precision for High VVR: {precision}')
print(f'Recall for High VVR: {recall}')
print(f'AUC-PR for High VVR: {auc_pr}')

Optimized Model Evaluation:
              precision    recall  f1-score   support

           0       0.85      0.95      0.90        43
           1       0.87      0.65      0.74        20

    accuracy                           0.86        63
   macro avg       0.86      0.80      0.82        63
weighted avg       0.86      0.86      0.85        63

Precision for High VVR: 0.8666666666666667
Recall for High VVR: 0.65
AUC-PR for High VVR: 0.8084839130403649


## With cluster

In [ ]:
# One-hot encode 'Cluster'
data_with_cluster = pd.get_dummies(data, columns=['Cluster'], prefix='Cluster')

# Check new columns
print(data_with_cluster.columns)

Index(['ID', 'Left Cheek Sum', 'Left Cheek Variance',
       'Left Cheek Standard Deviation', 'Left Cheek Maximum',
       'Left Cheek Minimum', 'Left Cheek Median', 'Left Cheek Mean',
       'Left Cheek Mean Absolute Change', 'Left Cheek Maximum Slope',
       'Left Cheek Minimum Slope', 'Right Cheek Sum', 'Right Cheek Variance',
       'Right Cheek Standard Deviation', 'Right Cheek Maximum',
       'Right Cheek Minimum', 'Right Cheek Median', 'Right Cheek Mean',
       'Right Cheek Mean Absolute Change', 'Right Cheek Maximum Slope',
       'Right Cheek Minimum Slope', 'Below Nose Sum', 'Below Nose Variance',
       'Below Nose Standard Deviation', 'Below Nose Maximum',
       'Below Nose Minimum', 'Below Nose Median', 'Below Nose Mean',
       'Below Nose Mean Absolute Change', 'Below Nose Maximum Slope',
       'Below Nose Minimum Slope', 'Between Eyes Sum', 'Between Eyes Variance',
       'Between Eyes Standard Deviation', 'Between Eyes Maximum',
       'Between Eyes Minimum', 'Bet

In [ ]:
# Include new cluster columns in the features list
features_with_cluster = data_with_cluster.columns.difference(['ID', 'Sum2']).tolist()  # Exclude 'ID' and 'Sum2' from features

# Define the target variable
target_with_cluster = 'Sum2'

In [ ]:
# Split the data into training and test sets
X_train_with_cluster, X_test_with_cluster, y_train_with_cluster, y_test_with_cluster = train_test_split(
    data_with_cluster[features_with_cluster], data_with_cluster[target_with_cluster], test_size=0.2, random_state=42)

from imblearn.over_sampling import SMOTE

# Apply SMOTE
smote_with_cluster = SMOTE(random_state=42)
X_train_smote_with_cluster, y_train_smote_with_cluster = smote_with_cluster.fit_resample(X_train_with_cluster, y_train_with_cluster)

In [ ]:
# Define the model
rf_with_cluster = RandomForestClassifier(random_state=42)

# Define the parameter grid
param_grid_with_cluster = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [10, 15, 20, 25, None],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 6]
}

# Setup the GridSearchCV
grid_search_with_cluster = GridSearchCV(estimator=rf_with_cluster, param_grid=param_grid_with_cluster, cv=5, scoring='recall', n_jobs=-1)

# Fit GridSearchCV
grid_search_with_cluster.fit(X_train_smote_with_cluster, y_train_smote_with_cluster)

# Best parameters and best recall
print("Best parameters including Cluster:", grid_search_with_cluster.best_params_)
print("Best recall including Cluster:", grid_search_with_cluster.best_score_)

Best parameters including Cluster: {'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 400}
Best recall including Cluster: 0.8620320855614974


In [ ]:
# Setup the optimized Random Forest model using the best parameters from GridSearchCV
optimized_rf_with_cluster = RandomForestClassifier(
    n_estimators=400,
    max_depth=15,
    min_samples_leaf=1,
    min_samples_split=2,
    random_state=42
)

# Train the model on the balanced training set
optimized_rf_with_cluster.fit(X_train_smote_with_cluster, y_train_smote_with_cluster)

# Predict on the test data
predictions_optimized_with_cluster = optimized_rf_with_cluster.predict(X_test_with_cluster)
probabilities_optimized_with_cluster = optimized_rf_with_cluster.predict_proba(X_test_with_cluster)[:, 1]

# Evaluate the model
print("Optimized Model Evaluation with Cluster:")
print(classification_report(y_test_with_cluster, predictions_optimized_with_cluster))

# Calculate specific metrics
precision_with_cluster = precision_score(y_test_with_cluster, predictions_optimized_with_cluster, pos_label=1)
recall_with_cluster = recall_score(y_test_with_cluster, predictions_optimized_with_cluster, pos_label=1)
auc_pr_with_cluster = average_precision_score(y_test_with_cluster, probabilities_optimized_with_cluster)
print(f'Precision for High VVR: {precision_with_cluster}')
print(f'Recall for High VVR: {recall_with_cluster}')
print(f'AUC-PR for High VVR: {auc_pr_with_cluster}')

Optimized Model Evaluation with Cluster:
              precision    recall  f1-score   support

           0       0.84      0.95      0.89        43
           1       0.86      0.60      0.71        20

    accuracy                           0.84        63
   macro avg       0.85      0.78      0.80        63
weighted avg       0.84      0.84      0.83        63

Precision for High VVR: 0.8571428571428571
Recall for High VVR: 0.6
AUC-PR for High VVR: 0.8276022702934247
